In [31]:
import os
file_path = "/Users/karthiksuresh/Desktop/CS762_project/github/CS762_Project/Notebooks"
result_files = []
for i in os.listdir(file_path):
    if i.endswith('_results.json'):
        result_files.append(i)

In [32]:
# with open("math_results1.json", 'r') as data_file:
#     math_results1 = json.load(data_file)
# with open("math_results2.json", 'r') as data_file:
#     math_results2 = json.load(data_file)

In [33]:
# math_results = math_results1 + math_results2
# with open('math_results.json', 'w') as json_file:
#     json.dump(math_results, json_file)

In [34]:
result_files

['hash_results.json',
 'matrix_results.json',
 'bitwise_results.json',
 'searching_results.json',
 'heap_results.json',
 'stack_results.json',
 'dp_results.json',
 'math_results.json',
 'pattern_results.json',
 'string_results.json',
 'array_results.json']

In [37]:
import json
combined = []
total=0
for f in result_files:
    result_path = os.path.join(file_path, f)
    with open(result_path, 'r') as data_file:
        results = json.load(data_file)
    combined.extend(results)
print(len(combined))

1334


In [38]:
with open('combined_results_v1_Dec5.json', 'w') as json_file:
    json.dump(combined, json_file)

In [107]:
with open('combined_results_v1_Dec5.json', 'r') as data_file:
    combined = json.load(data_file)

In [108]:
def remove_extra(input_string):
    index = input_string.find("###Explanation:")
    if index != -1:
        return input_string[:index]
    else:
        return input_string

In [110]:
import re
def parse_completion(completion):
    question_pattern = '(.*)###[ ]*Code:'
    code_pattern1 = '###[ ]*Code:.*```python(.*)```[\n]*###[ ]*Test[ ]*[c|C]ases:'
    code_pattern2 = '###[ ]*Code:(.*)[\n]*###[ ]*Test[ ]*[c|C]ases:'
    test_case_pattern1 = '###[ ]*Test[ ]*[c|C]ases:.*```python(.*)```[\n]*'
    test_case_pattern2 = '###[ ]*Test[ ]*[c|C]ases:(.*)'
#     print(completion)
    question_result = re.search(question_pattern, completion, re.DOTALL)
    if question_result:
        question = question_result.group(1).strip()
    else:
        question = 'Question not found'
    code_pattern1_result = re.search(code_pattern1, completion, re.DOTALL)
    code_pattern2_result = re.search(code_pattern2, completion, re.DOTALL)
    test_case_pattern1_result = re.search(test_case_pattern1, completion, re.DOTALL)
    test_case_pattern2_result = re.search(test_case_pattern2, completion, re.DOTALL)
    code=None
    test_cases=None
    if code_pattern1_result:
        code = code_pattern1_result.group(1).strip()
    elif code_pattern2_result:
        code = code_pattern2_result.group(1).strip()
    if test_case_pattern1_result:
        test_cases = remove_extra(test_case_pattern1_result.group(1).replace('[/Example 3]','').strip())
    elif test_case_pattern2_result:
        test_cases = remove_extra(test_case_pattern2_result.group(1).replace('[/Example 3]','').strip())
    test_case_list = re.findall('(assert.*)', test_cases)
    code_err = "Code Not Found"
    tc_err = "Test Cases not found"
    
    if not code and not test_cases:
        return code_err, tc_err, question
    elif not code:
        return code_err, test_cases, question
    elif not test_cases:
        return code, tc_err, question
    return code, test_case_list, question

In [112]:
import uuid
errors = []
for idx,i in enumerate(combined):
    code, tcs, question = parse_completion(i['completion'])
    if code=="Code Not Found" or tcs=="Test Cases not found" or question=='Question not found':
        errors.append(i['completion'])
    else:
#         combined[idx]['parsed'] = {'code':code, 'test_cases':tcs, 'question':question}
        combined[idx]['question'] = question
        combined[idx]['code'] = code
        combined[idx]['asserts'] = tcs
        combined[idx]['author'] = 'GPT'
        combined[idx]['UUID'] = str(uuid.uuid4())

In [120]:
combined_modified = []
for i in combined:
    if 'UUID' in i.keys():
        combined_modified.append(i)
print(len(combined))
print(len(combined_modified))

1334
1333


In [116]:
def pretty_print_parsed(p):
    tcs = "\n".join(p['asserts'])
    return f"Question:\n\n{p['question']}\n\nUUID:{p['UUID']}\n\nCode:\n\n{p['code']}\n\nAsserts\n\n{tcs}\n\n"

In [121]:
with open('combined_results_v1_Dec5.json', 'w') as json_file:
    json.dump(combined_modified, json_file)

In [19]:
import json
with open('combined_results_v1_Dec5.json', 'r') as data_file:
    combined = json.load(data_file)

In [20]:
def execute_python_code(code_string):
    try:
        exec(code_string, globals())
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        if e!=' ':
            print(e)
            print(code_string)
        print()
        return e

In [21]:
def prepend_imports(code_string):
    prefix = ['from typing import List','from typing import Dict']
    final_code=''
    for p in prefix:
        final_code = final_code+p+'\n'
    final_code+=code_string
    return final_code

In [22]:
from copy import deepcopy
import signal
combined_with_tc_status = []

timeout_seconds = 2
def handler(signum, frame):
    raise TimeoutError("Timeout occurred")

for idx in range(len(combined)):
    tc_status=[]
    print(idx)
    for tc in combined[idx]['asserts']:
        code = prepend_imports(combined[idx]['code'])
        code_string = code+'\n\n'+tc
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(timeout_seconds)
        try:
            error = execute_python_code(code_string)
        except TimeoutError as e:
            error = "Timeout Error"
        err = "Passed" if not error else str(error)
        tc_status.append((tc, err))
    record = deepcopy(combined[idx])
    record['tc_status'] = tc_status
    combined_with_tc_status.append(record)
#     with open('combined_with_tc_status_v1_Dec5.json', 'w') as json_file:
#         json.dump(combined_with_tc_status, json_file)

with open('combined_with_tc_status_v1_Dec5.json', 'w') as json_file:
    json.dump(combined_with_tc_status, json_file)

0
1
An error occurred: 

from typing import List
from typing import Dict
def uniquePairsCount(arr: list, target: int) -> int:
    unique_pairs = set()
    seen = set()
    for num in arr:
        complement = target - num
        if complement in seen:
            pair = (min(num, complement), max(num, complement))
            unique_pairs.add(pair)
        seen.add(num)
    
    return len(unique_pairs)

assert uniquePairsCount([1, 3, 2, 2, 4, 0, 5, 4], 5) == 2

An error occurred: 

from typing import List
from typing import Dict
def uniquePairsCount(arr: list, target: int) -> int:
    unique_pairs = set()
    seen = set()
    for num in arr:
        complement = target - num
        if complement in seen:
            pair = (min(num, complement), max(num, complement))
            unique_pairs.add(pair)
        seen.add(num)
    
    return len(unique_pairs)

assert uniquePairsCount([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 10) == 0

2
3
4
An error occurred: 

from typing import List
from typi

<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<string>:14: SyntaxWarning: "is" with a literal.

An error occurred: Timeout occurred
Timeout occurred
from typing import List
from typing import Dict
def bitwiseAND(a: int, b: int) -> int:
    while b:
        a, b = b, a & b
    return a

assert bitwiseAND(5, 7) == 5

An error occurred: Timeout occurred
Timeout occurred
from typing import List
from typing import Dict
def bitwiseAND(a: int, b: int) -> int:
    while b:
        a, b = b, a & b
    return a

assert bitwiseAND(6, 10) == 2

An error occurred: Timeout occurred
Timeout occurred
from typing import List
from typing import Dict
def bitwiseAND(a: int, b: int) -> int:
    while b:
        a, b = b, a & b
    return a

assert bitwiseAND(13, 19) == 1

An error occurred: Timeout occurred
Timeout occurred
from typing import List
from typing import Dict
def bitwiseAND(a: int, b: int) -> int:
    while b:
        a, b = b, a & b
    return a

assert bitwiseAND(8, 8) == 8

An error occurred: Timeout occurred
Timeout occurred
from typing import List
from typing import Dict
def bitwiseA


from typing import List
from typing import Dict
import heapq

def kClosest(points: list, k: int) -> list:
    heap = []
    for (x, y) in points:
        distance = x**2 + y**2
        heapq.heappush(heap, (distance, (x, y)))
    result = []
    for _ in range(k):
        result.append(heapq.heappop(heap)[1])
    return result

assert kClosest([[2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2], [1, 1]], 1) == [[1, 1]]

An error occurred: 

from typing import List
from typing import Dict
import heapq

def kClosest(points: list, k: int) -> list:
    heap = []
    for (x, y) in points:
        distance = x**2 + y**2
        heapq.heappush(heap, (distance, (x, y)))
    result = []
    for _ in range(k):
        result.append(heapq.heappop(heap)[1])
    return result

assert kClosest([[-1000000, -1000000], [1000000, 1000000]], 1) == [[-1000000, -1000000]]

335
An error occurred: 

from typing import List
from typing import Dict
import heapq

def kClosestElements(arr: list, k: int, x: int) -> 


from typing import List
from typing import Dict
def calculate(s: str) -> int:
    stack = []
    num = 0
    sign = 1
    result = 0
    for char in s:
        if char.isdigit():
            num = num * 10 + int(char)
        elif char in ['+', '-']:
            result += sign * num
            num = 0
            sign = 1 if char == '+' else -1
        elif char == '(':
            stack.append(result)
            stack.append(sign)
            result = 0
            sign = 1
        elif char == ')':
            result += sign * num
            num = 0
            result *= stack.pop()
            result += stack.pop()
    return result + sign * num

assert calculate("2*(5+5*2)/3+(6/2+8)")==21

An error occurred: 

from typing import List
from typing import Dict
def calculate(s: str) -> int:
    stack = []
    num = 0
    sign = 1
    result = 0
    for char in s:
        if char.isdigit():
            num = num * 10 + int(char)
        elif char in ['+', '-']:
            result +=

...

902
An error occurred: 

from typing import List
from typing import Dict
def hasRepeatingPattern(s: str) -> bool:
    n = len(s)
    for i in range(1, n//2 + 1):
        if n % i == 0:
            pattern = s[:i]
            if pattern * (n//i) == s:
                return True
    return False

assert hasRepeatingPattern("123123123") == False

903
904
905
906
907
An error occurred: 

from typing import List
from typing import Dict
def findAnagrams(s: str, p: str) -> List[int]:
    result = []
    p_count = Counter(p)
    s_count = Counter(s[:len(p)-1])
    for i in range(len(p)-1, len(s)):
        s_count[s[i]] += 1
        if s_count == p_count:
            result.append(i-len(p)+1)
        s_count[s[i-len(p)+1]] -= 1
        if s_count[s[i-len(p)+1]] == 0:
            del s_count[s[i-len(p)+1]]
    return result

assert findAnagrams("aabaabaa", "aaba") == [0, 1, 4]

An error occurred: 

from typing import List
from typing import Dict
def findAnagrams(s: str, p: str) -> List[int]:
  

An error occurred: 

from typing import List
from typing import Dict
def longest_palindrome(s: str) -> int:
    letter_counts = {}
    for char in s:
        letter_counts[char] = letter_counts.get(char, 0) + 1
    
    length = 0
    odd_found = False
    for count in letter_counts.values():
        if count % 2 == 0:
            length += count
        else:
            length += count - 1
            odd_found = True
    
    if odd_found:
        length += 1
    
    return length

assert longest_palindrome("ABACADAEAFAGAHAIAJA") == 1

An error occurred: 

from typing import List
from typing import Dict
def longest_palindrome(s: str) -> int:
    letter_counts = {}
    for char in s:
        letter_counts[char] = letter_counts.get(char, 0) + 1
    
    length = 0
    odd_found = False
    for count in letter_counts.values():
        if count % 2 == 0:
            length += count
        else:
            length += count - 1
            odd_found = True
    
    if odd_found:
       


from typing import List
from typing import Dict
def count_substrings_with_abc(s: str) -> int:
    """
    Counts the number of substrings containing 'a', 'b', and 'c' exactly once.
    
    Parameters:
    s (str): The input string consisting of characters 'a', 'b', and 'c' only.
    
    Returns:
    int: The number of substrings that contain 'a', 'b', and 'c' exactly once.
    """
    counts = { 'a': 0, 'b': 0, 'c': 0 }
    left = 0
    result = 0
    
    for right, char in enumerate(s):
        if char in counts:
            counts[char] += 1
        
        while all(counts.values()):
            counts[s[left]] -= 1
            left += 1
        
        if counts['a'] == 1 and counts['b'] == 1 and counts['c'] == 1:
            result += left

    return result

assert count_substrings_with_abc('aaabbbcccabc') == 18  # Mix of multiple substrings

An error occurred: 

from typing import List
from typing import Dict
def count_substrings_with_abc(s: str) -> int:
    """
    Counts

An error occurred: 'type' object is not subscriptable
'type' object is not subscriptable
from typing import List
from typing import Dict
def three_sum(nums: list[int]) -> list[list[int]]:
    nums.sort()
    result = []
    for i in range(len(nums) - 2):
        if i > 0 and nums[i] == nums[i-1]:
            continue
        left, right = i + 1, len(nums) - 1
        while left < right:
            sum = nums[i] + nums[left] + nums[right]
            if sum < 0:
                left += 1
            elif sum > 0:
                right -= 1
            else:
                result.append([nums[i], nums[left], nums[right]])
                while left < right and nums[left] == nums[left + 1]:
                    left += 1
                while left < right and nums[right] == nums[right - 1]:
                    right -= 1
                left += 1
                right -= 1
    return result

assert three_sum([3, -2, 1, 0]) == []

An error occurred: 'type' object is not subscriptable
'typ

An error occurred: 

from typing import List
from typing import Dict
import numpy as np
def findMaxAverage(nums: list, k: int) -> float:
    if len(nums) == 0 or k <= 0 or k > len(nums):
        return 0
    current_sum = sum(nums[0:k])
    max_sum = current_sum
    for i in range(len(nums) - k):
        current_sum = current_sum - nums[i] + nums[i+k]
        max_sum = max(max_sum, current_sum)
    return max_sum / k

assert findMaxAverage([4, 0, 4, 3, 3, 1, -1, 5], 5) == 3.0

An error occurred: 

from typing import List
from typing import Dict
import numpy as np
def findMaxAverage(nums: list, k: int) -> float:
    if len(nums) == 0 or k <= 0 or k > len(nums):
        return 0
    current_sum = sum(nums[0:k])
    max_sum = current_sum
    for i in range(len(nums) - k):
        current_sum = current_sum - nums[i] + nums[i+k]
        max_sum = max(max_sum, current_sum)
    return max_sum / k

assert findMaxAverage([-1, -2, -3, -4], 2) == -2.5

An error occurred: 

from typing import List

In [18]:
import json
with open('combined_with_tc_status_v1_Dec5.json', 'r') as data_file:
    combined_with_tc_status = json.load(data_file)

In [19]:
len(combined_with_tc_status)

1333

In [20]:
asserts_passed = 0
asserts_failed = 0
asserts_failed_indices={}
for idx,i in enumerate(combined_with_tc_status):
    for j in i['tc_status']:
        if j[1]=='Passed':
            asserts_passed+=1
            
        else:
            asserts_failed+=1
            if str(idx) not in asserts_failed_indices:
                asserts_failed_indices[str(idx)] = {'failed':1, 'total':len(i['tc_status'])}
            else:
                asserts_failed_indices[str(idx)]['failed']+=1
print(round(asserts_passed/(asserts_passed+asserts_failed), 2))
print(asserts_failed_indices)

0.81
{'1': {'failed': 2, 'total': 6}, '4': {'failed': 1, 'total': 8}, '6': {'failed': 3, 'total': 5}, '7': {'failed': 2, 'total': 7}, '10': {'failed': 1, 'total': 6}, '16': {'failed': 4, 'total': 6}, '17': {'failed': 1, 'total': 6}, '18': {'failed': 5, 'total': 6}, '21': {'failed': 1, 'total': 3}, '24': {'failed': 4, 'total': 7}, '25': {'failed': 2, 'total': 4}, '27': {'failed': 5, 'total': 5}, '28': {'failed': 3, 'total': 6}, '35': {'failed': 1, 'total': 3}, '36': {'failed': 1, 'total': 5}, '39': {'failed': 1, 'total': 6}, '42': {'failed': 1, 'total': 4}, '43': {'failed': 2, 'total': 5}, '45': {'failed': 4, 'total': 6}, '48': {'failed': 2, 'total': 7}, '49': {'failed': 4, 'total': 6}, '50': {'failed': 4, 'total': 5}, '52': {'failed': 1, 'total': 6}, '53': {'failed': 4, 'total': 7}, '54': {'failed': 2, 'total': 5}, '56': {'failed': 4, 'total': 4}, '57': {'failed': 1, 'total': 8}, '58': {'failed': 6, 'total': 6}, '59': {'failed': 7, 'total': 7}, '64': {'failed': 2, 'total': 5}, '65': {'

In [21]:
avg_fail = []
for i in asserts_failed_indices:
    fails = asserts_failed_indices[i]['failed']
    total = asserts_failed_indices[i]['total']
    avg_fail.append(fails/total)
total_avg_fail = sum(avg_fail)/len(avg_fail)
print(round(total_avg_fail,3))

0.434


In [23]:
asserts_majority_fail = {}
for i in asserts_failed_indices:
    if asserts_failed_indices[i]['failed'] >= asserts_failed_indices[i]['total']//2:
        asserts_majority_fail[i] = asserts_failed_indices[i]
len(asserts_majority_fail)

240

In [25]:
from copy import deepcopy
# Include only those that dont have majoritarily failing cases
combined_copy = []
for idx, i in enumerate(combined_with_tc_status):
    if str(idx) in asserts_majority_fail:
        continue
    if str(idx) in asserts_failed_indices:
        record = i
        new_asserts = []
        new_tc_status = []
        for j in record['tc_status']:
            if j[1]=='Passed':
                new_tc_status.append(j)
                new_asserts.append(j[0])
        record['asserts'] = new_asserts
        record['tc_status'] = new_tc_status
        combined_copy.append(record)
    else:
        combined_copy.append(i)

In [26]:
len(combined_copy)

1093

In [28]:
with open('combined_v1_Dec5_after_exec_check.json', 'w') as json_file:
    json.dump(combined_copy, json_file)

In [1]:
import json
with open('combined_v1_Dec5_after_exec_check.json', 'r') as data_file:
    combined = json.load(data_file)

In [2]:
from rouge_score import rouge_scorer

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('e5-base-v2')
input_texts = [
    'query: how much protein should a female eat',
    'query: summit define',
    "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
]
embeddings = model.encode(input_texts, normalize_embeddings=True)


In [22]:
def calculate_rouge_similarity(strings):
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    similarity_matrix = {}

    for i, hypothesis in strings:
        print(i)
        for j, reference in strings:
            if j>i:
                key = (i, j)
                scores = rouge_scorer_instance.score(hypothesis, reference)
                scores = 
                similarity_matrix[key] = scores

    return similarity_matrix

In [23]:
question_strings = []
for idx, i in enumerate(combined):
    question_strings.append((idx, i['question']))

In [29]:
sim_matrix = calculate_rouge_similarity(question_strings)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [30]:
with open('sim_matrix_v1_Dec5.json', 'w') as json_file:
    json.dump(sim_matrix, json_file)

TypeError: keys must be str, int, float, bool or None, not tuple

In [51]:
combined[0]['question']

'Given a list of strings, write a function to group the strings by their anagrams. Anagrams are the strings which have the same characters but may be in a different order. Return a list of lists, where each inner list contains strings that are anagrams of each other.'

In [39]:
from copy import deepcopy
sim_matrix_copy = deepcopy(sim_matrix)

In [65]:
import time
counter=0
for idx,i in enumerate(sim_matrix_copy):
    if idx%1000==0:
#         time.sleep(1)
        print(idx)
    rouge_s = sim_matrix_copy[i]["rougeL"].fmeasure
    if rouge_s>0.3:
        counter+=1
#     if combined[i[0]]['category']==combined[i[1]]['category'] and rouge_s>0.4:
#         print(f'Question 1:\n\n{combined[i[0]]["question"]}\n\nQuestion 2:\n\n{combined[i[1]]["question"]}\n\nRouge-L:\n{rouge_s}')
print(counter)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [64]:
len(sim_matrix_copy)*2

1193556